# Testing two different node operations for the CNN RandDense model
See Section 6b of the manuscript

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/USC Random NN')

Mounted at /content/drive


In [ ]:
!pip install eofs --quiet
import numpy as np
import xarray as xr
import pandas as pd
from utils import *
from utils_cnn import *

from random_nn import *
from random_nn_alt_node_ops import *
import keras
from keras import Model
from keras.layers import *
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.utils import plot_model
tf.get_logger().setLevel('ERROR')
from IPython.display import display

tf.keras.utils.set_random_seed(21)

     |████████████████████████████████| 1.0 MB 4.8 MB/s 


In [ ]:
vars_to_predict = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']
simus = ['historical', 'hist-GHG', 'hist-aer', 'ssp126', 'ssp370', 'ssp585',]
# Selects first two years of every decade from 1850 onward as validation
val_idx = np.concatenate((np.arange(0,161,10), np.arange(1,162,10),
                          np.arange(165,326,10), np.arange(166,327,10), 
                          np.arange(330,491,10), np.arange(331,492,10),
                          np.arange(500,571,10), np.arange(501,572,10),
                          np.arange(586,657,10), np.arange(587,658,10),
                          np.arange(672,743,10), np.arange(673,744,10)))

X_train_dict = {}
Y_train_dict = {}
X_val_dict = {}
Y_val_dict = {}

# Create training data
for var in vars_to_predict:
  X, Y, meanstd_inputs = create_training_data(simus, var_to_predict=var)
    
  X_val = np.take(X, val_idx, axis=0)
  X_train = np.delete(X, val_idx, axis=0)
  Y_val = np.take(Y, val_idx, axis=0)
  Y_train = np.delete(Y, val_idx, axis=0)
    
  X_train_dict[var] = X_train
  X_val_dict[var] = X_val
  Y_train_dict[var] = Y_train
  Y_val_dict[var] = Y_val

# Open, reformat, and normalize test data
X_test = xr.open_mfdataset([data_path + 'inputs_historical.nc',
                            data_path + 'inputs_ssp245.nc']).compute()
Y_test = create_predictdand_data(['ssp245'])

for input_var in ['CO2', 'CH4', 'SO2', 'BC']: 
  var_dims = X_test[input_var].dims
  X_test = X_test.assign({input_var: (var_dims, normalize(X_test[input_var].data, input_var, meanstd_inputs))}) 
    
X_test_np = input_for_training(X_test) 

X_train_dict['tas'].shape, Y_train_dict['tas'].shape, X_val_dict['tas'].shape, Y_val_dict['tas'].shape

((603, 96, 144, 4), (603, 96, 144), (150, 96, 144, 4), (150, 96, 144))

In [ ]:
param_lims = {1000000: '1M',
              10000000: '10M'
             }
layer_range = (2,11)
num_models = 10
raw_rmse_data_spatial = []
raw_rmse_data_global = []
raw_rmse_data_total = []
results_path = results_path + 'cnn_rand_dense/node_op_experiments/relu_position/'

In [ ]:
for param_lim in param_lims.keys():
    
  # param_lim +/- 10%
  param_range = (int(param_lim-0.1*param_lim), int(param_lim+0.1*param_lim))
  rmse_data_spatial = []
  rmse_data_global = []
  rmse_data_total = []
  adj_mat_data = []

  for num_layers in range(*layer_range):

    for i in range(num_models):
            
      keras.backend.clear_session()
      untrained_model = None

      model_input = Input(shape=(96,144,4,))
      x = Conv2D(20, (3,3), padding='same', activation='relu', kernel_regularizer='l2')(model_input)
      x = AveragePooling2D(2)(x)
      x = GlobalAveragePooling2D()(x)
      x, adj_mat, param_count = RandDenseNodeOps(prev_layer=x,
                                                 out_shape=96*144,
                                                 layer_count_range=(num_layers,num_layers),
                                                 layer_size_range=(x.shape[1],96*144),
                                                 param_lim=param_range,
                                                 relu_position='after')
      model_output = Reshape((1, 96, 144))(x)
      untrained_model = Model(model_input, model_output)

      #untrained_model.summary()
      #display(plot_model(untrained_model, show_shapes=True))

      np.save(results_path+f'{param_lims[param_lim]}/models/{num_layers}_layer_model_{i}', untrained_model.get_config())
      plot_model(untrained_model, results_path+f'{param_lims[param_lim]}/images/{num_layers}_layer_model_{i}.png', show_shapes=True)
      adj_mat_data.append(adj_mat)
            
      for var in vars_to_predict:
                
        # Get train/val data
        X_train = X_train_dict[var]
        Y_train = Y_train_dict[var]
        X_val = X_val_dict[var]
        Y_val = Y_val_dict[var]

        model = None
        model = untrained_model
        model.compile(optimizer="adam", loss="mse", metrics=["mse"])
        hist = model.fit(X_train,
                         Y_train,
                         batch_size=16,
                         epochs=100,
                         verbose=0,
                         validation_data=(X_val,Y_val),
                         callbacks=EarlyStopping(patience=10, restore_best_weights=True),
                        )
        #plot_loss(hist)
                
        # Make predictions using trained model
        m_pred = model.predict(X_test_np)
        m_pred = m_pred.reshape(m_pred.shape[0], m_pred.shape[2], m_pred.shape[3])
        m_pred = xr.DataArray(m_pred, dims=['time','lat','lon'], coords=[X_test.time.data, X_test.latitude.data, X_test.longitude.data])
        m_pred = m_pred.transpose('lat','lon','time').sel(time=slice(2015,2101)).to_dataset(name=var)

        # Save prediction as .nc
        '''
        if var == 'diurnal_temperature_range':
          m_pred.to_netcdf(results_path+f'{param_lims[param_lim]}/predictions/dtr/{num_layers}_layer_model_{i}.nc', 'w')
        else:
          m_pred.to_netcdf(results_path+f'{param_lims[param_lim]}/predictions/{var}/{num_layers}_layer_model_{i}.nc', 'w')
        '''

        # Calculate RMSE
        var_truth = Y_test[var]
        m_var_pred = m_pred.transpose('time','lat','lon')[var]
        rmse_spatial = get_rmse_spatial(var_truth[65:], m_var_pred[65:])
        raw_rmse_data_spatial.append(rmse_spatial)
        rmse_global = get_rmse_global(var_truth[65:], m_var_pred[65:])
        raw_rmse_data_global.append(rmse_global)
        rmse_total = rmse_spatial + 5*rmse_global
        raw_rmse_data_total.append(rmse_total)
                
    n_layer_rmse_data_spatial = raw_rmse_data_spatial[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_spatial = np.reshape(n_layer_rmse_data_spatial, (len(vars_to_predict), num_models), order='F')

    n_layer_rmse_data_global = raw_rmse_data_global[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_global = np.reshape(n_layer_rmse_data_global, (len(vars_to_predict), num_models), order='F')

    n_layer_rmse_data_total = raw_rmse_data_total[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_total = np.reshape(n_layer_rmse_data_total, (len(vars_to_predict), num_models), order='F')

    print(f'{num_layers} Layer {param_lims[param_lim]} Mean RMSE:')
    for var in vars_to_predict:
        if var == 'diurnal_temperature_range':
            print(f'\tdtr:')
            print(f'\t\tSpatial: {round(np.mean(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(np.mean(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(np.mean(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
        else:
            print(f'\t{var}:')
            print(f'\t\tSpatial: {round(np.mean(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(np.mean(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(np.mean(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')

    print(f'{num_layers} Layer {param_lims[param_lim]} Min RMSE:')
    for var in vars_to_predict:
        if var == 'diurnal_temperature_range':
            print(f'\tdtr:')
            print(f'\t\tSpatial: {round(min(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(min(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(min(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
        else:
            print(f'\t{var}:')
            print(f'\t\tSpatial: {round(min(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(min(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(min(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
            
    rmse_data_spatial.append(n_layer_rmse_data_spatial)
    rmse_data_global.append(n_layer_rmse_data_global)
    rmse_data_total.append(n_layer_rmse_data_total)
        
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_spatial', rmse_data_spatial)
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_global', rmse_data_global)
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_total', rmse_data_total)
  adj_mat_data = np.array(adj_mat_data, dtype='object')
  adj_mat_data = np.reshape(adj_mat_data, (len(range(*layer_range)), num_models))
  np.save(results_path+f'{param_lims[param_lim]}/adj_mat_data', adj_mat_data)

2 Layer 1M Mean RMSE:
	tas:
		Spatial: 0.7479
		Global: 0.6525
		Total: 4.0106
	dtr:
		Spatial: 17.0888
		Global: 1.433
		Total: 24.2538
	pr:
		Spatial: 5.2924
		Global: 0.8682
		Total: 9.6335
	pr90:
		Spatial: 6.2635
		Global: 0.8874
		Total: 10.7006
2 Layer 1M Min RMSE:
	tas:
		Spatial: 0.7005
		Global: 0.6165
		Total: 3.7831
	dtr:
		Spatial: 16.8857
		Global: 1.3287
		Total: 23.7116
	pr:
		Spatial: 5.1911
		Global: 0.844
		Total: 9.4646
	pr90:
		Spatial: 6.1769
		Global: 0.8685
		Total: 10.5796
3 Layer 1M Mean RMSE:
	tas:
		Spatial: 0.8415
		Global: 0.7329
		Total: 4.5058
	dtr:
		Spatial: 16.9994
		Global: 1.4022
		Total: 24.0104
	pr:
		Spatial: 5.3025
		Global: 0.8575
		Total: 9.5901
	pr90:
		Spatial: 6.2586
		Global: 0.8922
		Total: 10.7194
3 Layer 1M Min RMSE:
	tas:
		Spatial: 0.8149
		Global: 0.7095
		Total: 4.3621
	dtr:
		Spatial: 16.7616
		Global: 1.3439
		Total: 23.737
	pr:
		Spatial: 5.1107
		Global: 0.8272
		Total: 9.2466
	pr90:
		Spatial: 6.1854
		Global: 0.8815
		Total: 1

In [ ]:
from utils import results_path

param_lims = {1000000: '1M',
              10000000: '10M'
             }
layer_range = (2,11)
num_models = 10
raw_rmse_data_spatial = []
raw_rmse_data_global = []
raw_rmse_data_total = []
results_path = results_path + 'cnn_rand_dense/node_op_experiments/weighted_sum/'

In [ ]:
for param_lim in param_lims.keys():
    
  # param_lim +/- 10%
  param_range = (int(param_lim-0.1*param_lim), int(param_lim+0.1*param_lim))
  rmse_data_spatial = []
  rmse_data_global = []
  rmse_data_total = []
  adj_mat_data = []

  for num_layers in range(*layer_range):

    for i in range(num_models):
            
      keras.backend.clear_session()
      untrained_model = None

      model_input = Input(shape=(96,144,4,))
      x = Conv2D(20, (3,3), padding='same', activation='relu', kernel_regularizer='l2')(model_input)
      x = AveragePooling2D(2)(x)
      x = GlobalAveragePooling2D()(x)
      x, adj_mat, param_count = RandDenseNodeOps(prev_layer=x,
                                                 out_shape=96*144,
                                                 layer_count_range=(num_layers,num_layers),
                                                 layer_size_range=(x.shape[1],96*144),
                                                 param_lim=param_range,
                                                 weighted_sum=False)
      model_output = Reshape((1, 96, 144))(x)
      untrained_model = Model(model_input, model_output)

      #untrained_model.summary()
      #display(plot_model(untrained_model, show_shapes=True))

      np.save(results_path+f'{param_lims[param_lim]}/models/{num_layers}_layer_model_{i}', untrained_model.get_config())
      plot_model(untrained_model, results_path+f'{param_lims[param_lim]}/images/{num_layers}_layer_model_{i}.png', show_shapes=True)
      adj_mat_data.append(adj_mat)
            
      for var in vars_to_predict:
                
        # Get train/val data
        X_train = X_train_dict[var]
        Y_train = Y_train_dict[var]
        X_val = X_val_dict[var]
        Y_val = Y_val_dict[var]

        model = None
        model = untrained_model
        model.compile(optimizer="adam", loss="mse", metrics=["mse"])
        hist = model.fit(X_train,
                         Y_train,
                         batch_size=16,
                         epochs=100,
                         verbose=0,
                         validation_data=(X_val,Y_val),
                         callbacks=EarlyStopping(patience=10, restore_best_weights=True),
                        )
        #plot_loss(hist)
                
        # Make predictions using trained model
        m_pred = model.predict(X_test_np)
        m_pred = m_pred.reshape(m_pred.shape[0], m_pred.shape[2], m_pred.shape[3])
        m_pred = xr.DataArray(m_pred, dims=['time','lat','lon'], coords=[X_test.time.data, X_test.latitude.data, X_test.longitude.data])
        m_pred = m_pred.transpose('lat','lon','time').sel(time=slice(2015,2101)).to_dataset(name=var)

        # Save prediction as .nc
        '''
        if var == 'diurnal_temperature_range':
          m_pred.to_netcdf(results_path+f'{param_lims[param_lim]}/predictions/dtr/{num_layers}_layer_model_{i}.nc', 'w')
        else:
          m_pred.to_netcdf(results_path+f'{param_lims[param_lim]}/predictions/{var}/{num_layers}_layer_model_{i}.nc', 'w')
        '''

        # Calculate RMSE
        var_truth = Y_test[var]
        m_var_pred = m_pred.transpose('time','lat','lon')[var]
        rmse_spatial = get_rmse_spatial(var_truth[65:], m_var_pred[65:])
        raw_rmse_data_spatial.append(rmse_spatial)
        rmse_global = get_rmse_global(var_truth[65:], m_var_pred[65:])
        raw_rmse_data_global.append(rmse_global)
        rmse_total = rmse_spatial + 5*rmse_global
        raw_rmse_data_total.append(rmse_total)
                
    n_layer_rmse_data_spatial = raw_rmse_data_spatial[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_spatial = np.reshape(n_layer_rmse_data_spatial, (len(vars_to_predict), num_models), order='F')

    n_layer_rmse_data_global = raw_rmse_data_global[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_global = np.reshape(n_layer_rmse_data_global, (len(vars_to_predict), num_models), order='F')

    n_layer_rmse_data_total = raw_rmse_data_total[-(num_models*len(vars_to_predict)):]
    n_layer_rmse_data_total = np.reshape(n_layer_rmse_data_total, (len(vars_to_predict), num_models), order='F')

    print(f'{num_layers} Layer {param_lims[param_lim]} Mean RMSE:')
    for var in vars_to_predict:
        if var == 'diurnal_temperature_range':
            print(f'\tdtr:')
            print(f'\t\tSpatial: {round(np.mean(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(np.mean(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(np.mean(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
        else:
            print(f'\t{var}:')
            print(f'\t\tSpatial: {round(np.mean(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(np.mean(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(np.mean(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')

    print(f'{num_layers} Layer {param_lims[param_lim]} Min RMSE:')
    for var in vars_to_predict:
        if var == 'diurnal_temperature_range':
            print(f'\tdtr:')
            print(f'\t\tSpatial: {round(min(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(min(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(min(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
        else:
            print(f'\t{var}:')
            print(f'\t\tSpatial: {round(min(n_layer_rmse_data_spatial[vars_to_predict.index(var)]),4)}')
            print(f'\t\tGlobal: {round(min(n_layer_rmse_data_global[vars_to_predict.index(var)]),4)}')
            print(f'\t\tTotal: {round(min(n_layer_rmse_data_total[vars_to_predict.index(var)]),4)}')
            
    rmse_data_spatial.append(n_layer_rmse_data_spatial)
    rmse_data_global.append(n_layer_rmse_data_global)
    rmse_data_total.append(n_layer_rmse_data_total)
        
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_spatial', rmse_data_spatial)
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_global', rmse_data_global)
  np.save(results_path+f'{param_lims[param_lim]}/rmse_data_total', rmse_data_total)
  adj_mat_data = np.array(adj_mat_data, dtype='object')
  adj_mat_data = np.reshape(adj_mat_data, (len(range(*layer_range)), num_models))
  np.save(results_path+f'{param_lims[param_lim]}/adj_mat_data', adj_mat_data)

2 Layer 1M Mean RMSE:
	tas:
		Spatial: 0.7552
		Global: 0.6621
		Total: 4.0657
	dtr:
		Spatial: 17.1924
		Global: 1.3904
		Total: 24.1444
	pr:
		Spatial: 5.2956
		Global: 0.8797
		Total: 9.6939
	pr90:
		Spatial: 6.2489
		Global: 0.8949
		Total: 10.7235
2 Layer 1M Min RMSE:
	tas:
		Spatial: 0.7063
		Global: 0.6218
		Total: 3.8154
	dtr:
		Spatial: 16.9707
		Global: 1.2938
		Total: 23.7799
	pr:
		Spatial: 5.2104
		Global: 0.8467
		Total: 9.523
	pr90:
		Spatial: 6.1578
		Global: 0.886
		Total: 10.5879
3 Layer 1M Mean RMSE:
	tas:
		Spatial: 0.7716
		Global: 0.6742
		Total: 4.1427
	dtr:
		Spatial: 17.047
		Global: 1.4356
		Total: 24.225
	pr:
		Spatial: 5.2734
		Global: 0.8768
		Total: 9.6575
	pr90:
		Spatial: 6.1741
		Global: 0.8873
		Total: 10.6106
3 Layer 1M Min RMSE:
	tas:
		Spatial: 0.7092
		Global: 0.6183
		Total: 3.8007
	dtr:
		Spatial: 16.4538
		Global: 1.309
		Total: 23.7242
	pr:
		Spatial: 5.1733
		Global: 0.8345
		Total: 9.4031
	pr90:
		Spatial: 6.0295
		Global: 0.8756
		Total: 10.